# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [3]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [4]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

loan_binary_encoded = pd.get_dummies(df, columns=["home_ownership", "verification_status", "pymnt_plan", 
                                                  "hardship_flag", "debt_settlement_flag", "initial_list_status", 
                                                 "next_pymnt_d", "application_type"])
#from sklearn.preprocessing import LabelEncoder
#le = LabelEncoder()
#df2 = loan_binary_encoded.copy()
#df2['loan_status'] = le.fit_transform(df2['loan_status'])

loan_binary_encoded.head()

,loan_amnt,int_rate,installment,annual_inc,issue_d,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,...,verification_status_Verified,pymnt_plan_n,hardship_flag_N,debt_settlement_flag_N,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App
0,10500.0,0.1719,375.35,66000.0,Mar-2019,low_risk,27.24,0.0,0.0,8.0,...,0,1,1,1,0,1,0,1,1,0
1,25000.0,0.2000,929.09,105000.0,Mar-2019,low_risk,20.23,0.0,0.0,17.0,...,1,1,1,1,0,1,0,1,1,0
2,20000.0,0.2000,529.88,56000.0,Mar-2019,low_risk,24.26,0.0,0.0,8.0,...,1,1,1,1,0,1,0,1,1,0
3,10000.0,0.1640,353.55,92000.0,Mar-2019,low_risk,31.44,0.0,1.0,10.0,...,1,1,1,1,0,1,0,1,1,0
4,22000.0,0.1474,520.39,52000.0,Mar-2019,low_risk,18.76,0.0,1.0,14.0,...,0,1,1,1,0,1,0,1,1,0


# Split the Data into Training and Testing

In [5]:
# Create our features
X = loan_binary_encoded.drop(columns=["loan_status", "issue_d"])
# Create our target
y = loan_binary_encoded["loan_status"]

In [6]:
X.dtypes[X.dtypes != 'int64'][X.dtypes != 'float64']

home_ownership_ANY                     uint8
home_ownership_MORTGAGE                uint8
home_ownership_OWN                     uint8
home_ownership_RENT                    uint8
verification_status_Not Verified       uint8
verification_status_Source Verified    uint8
verification_status_Verified           uint8
pymnt_plan_n                           uint8
hardship_flag_N                        uint8
debt_settlement_flag_N                 uint8
initial_list_status_f                  uint8
initial_list_status_w                  uint8
next_pymnt_d_Apr-2019                  uint8
next_pymnt_d_May-2019                  uint8
application_type_Individual            uint8
application_type_Joint App             uint8
dtype: object

In [7]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
X_train.shape

(51612, 92)

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [9]:
# Resample the training data with the RandomOversampler

from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({'low_risk': 51366, 'high_risk': 51366})

In [10]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [11]:
# Calculated the balanced accuracy score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score

y_pred = model.predict(X_test)

balanced_accuracy_score(y_test, y_pred)

0.6676404222508312

In [12]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)


array([[   74,    27],
       [ 6797, 10307]], dtype=int64)

In [13]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.73      0.60      0.02      0.66      0.45       101
   low_risk       1.00      0.60      0.73      0.75      0.66      0.44     17104

avg / total       0.99      0.60      0.73      0.75      0.66      0.44     17205



### SMOTE Oversampling

In [15]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
XSMOTE_resampled, ySMOTE_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(
    X_train, y_train
)
Counter(ySMOTE_resampled)

Counter({'low_risk': 51366, 'high_risk': 51366})

In [16]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(XSMOTE_resampled, ySMOTE_resampled)

LogisticRegression(random_state=1)

In [17]:
# Calculated the balanced accuracy score
ySMOTE_pred = model.predict(X_test)
balanced_accuracy_score(y_test, ySMOTE_pred)

0.6629113449230798

In [18]:
# Display the confusion matrix
confusion_matrix(y_test, ySMOTE_pred)

array([[   66,    35],
       [ 5604, 11500]], dtype=int64)

In [19]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, ySMOTE_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.65      0.67      0.02      0.66      0.44       101
   low_risk       1.00      0.67      0.65      0.80      0.66      0.44     17104

avg / total       0.99      0.67      0.65      0.80      0.66      0.44     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [20]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
XCLCN_resampled, yCLCN_resampled = cc.fit_resample(X_train, y_train)
Counter(yCLCN_resampled)

Counter({'high_risk': 246, 'low_risk': 246})

In [21]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=78)
model.fit(XCLCN_resampled, yCLCN_resampled)

LogisticRegression(random_state=78)

In [24]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, yCLCN_pred)

0.5315880599987033

In [23]:
# Display the confusion matrix
yCLCN_pred = model.predict(X_test)
confusion_matrix(y_test, yCLCN_pred)

array([[   68,    33],
       [10435,  6669]], dtype=int64)

In [25]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, yCLCN_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.67      0.39      0.01      0.51      0.27       101
   low_risk       1.00      0.39      0.67      0.56      0.51      0.26     17104

avg / total       0.99      0.39      0.67      0.56      0.51      0.26     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [26]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=0)
XSMOTEENN_resampled, ySMOTEENN_resampled = smote_enn.fit_resample(X, y)
Counter(ySMOTEENN_resampled)

Counter({'high_risk': 68460, 'low_risk': 62011})

In [27]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(XSMOTEENN_resampled, ySMOTEENN_resampled)

LogisticRegression(random_state=1)

In [29]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, ySMOTEENN_pred)

0.6549533315118228

In [28]:
# Display the confusion matrix
ySMOTEENN_pred = model.predict(X_test)
confusion_matrix(y_test, ySMOTEENN_pred)

array([[  74,   27],
       [7231, 9873]], dtype=int64)

In [30]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, ySMOTEENN_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.73      0.58      0.02      0.65      0.43       101
   low_risk       1.00      0.58      0.73      0.73      0.65      0.42     17104

avg / total       0.99      0.58      0.73      0.73      0.65      0.42     17205

